# Circuit simulation examples

In [1]:
import obi_one as obi
from pathlib import Path

In [2]:
circuit_path_prefix = Path("../../data/tiny_circuits")

### Loading two circuits

In [3]:
circuit_name = "N_10__top_nodes_dim6"
circuit = obi.Circuit(name=circuit_name, path=str(circuit_path_prefix / circuit_name / "circuit_config.json"))
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes[circuit.default_population_name].size} neurons and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")

Circuit 'N_10__top_nodes_dim6' with 10 neurons and 176 synapses


In [4]:
circuit2_name = "N_10__top_rc_nodes_dim2_rc"
circuit2 = obi.Circuit(name=circuit2_name, path=str(circuit_path_prefix / circuit2_name / "circuit_config.json"))
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes[circuit2.default_population_name].size} neurons and {circuit2.sonata_circuit.edges[circuit2.default_edge_population_name].size} synapses")

Circuit 'N_10__top_rc_nodes_dim2_rc' with 10 neurons and 19 synapses


### Example with two circuits and explicit node populations.

Spike replay will use POm.

In [5]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
node_pop = circuit._default_population_name(circuit.sonata_circuit)

sim_neuron_set = obi.AllNeurons()
stim_neuron_set = sim_neuron_set
replay_neuron_set = sim_neuron_set
rec_neuron_set = sim_neuron_set
property_neuron_set = sim_neuron_set

# sim_neuron_set = obi.PredefinedNeuronSet(node_set="All", node_population=node_pop)
# stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100], node_population=node_pop)
# replay_neuron_set = obi.PredefinedNeuronSet(node_set="proj_Thalamocortical_POM_Source", sample_percentage=25, node_population="POm")
# rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100, node_population=node_pop)
# property_neuron_set = obi.PropertyNeuronSet(
#     property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
#                      obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})],
#     node_population=node_pop
# )
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="POM_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [6]:
# simulations_form.model_dump(mode="json")

In [7]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_explicit')
grid_scan.multiple_value_parameters(display=True)
# grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute()
for single_config in grid_scan.single_configs:
    task_type = obi.get_configs_task_type(single_config)
    task = task_type(config=single_config)
    task.execute()

[2025-09-24 12:23:24,983] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-09-24 12:23:24,983] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-09-24 12:23:24,983] INFO: initialize.circuit: [Circuit(name='N_10__top_nodes_dim6', path='../../data/tiny_circuits/N_10__top_nodes_dim6/circuit_config.json', matrix_path=None, type='Circuit'), Circuit(name='N_10__top_rc_nodes_dim2_rc', path='../../data/tiny_circuits/N_10__top_rc_nodes_dim2_rc/circuit_config.json', matrix_path=None, type='Circuit')]
[2025-09-24 12:23:24,989] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 12:23:25,025] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 12:23:25,060] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 12:23:25,095] INFO: initialize.circuit is a Circuit instance.


In [8]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_explicit/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.StimNeurons.sample_percentage=50.0/neuron_sets.PropertyNeuronSet.property_filter=layer=1,2,3,synapse_class=INH/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/POm"]["node_ids"][:])
h5.close()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../../../../obi-output/circuit_simulations/grid_scan_explicit/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.StimNeurons.sample_percentage=50.0/neuron_sets.PropertyNeuronSet.property_filter=layer=1,2,3,synapse_class=INH/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### Implicit node populations, i.e. using the default.

Spike replay will use a subset of the default population, in this case: Layer2

In [ ]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
sim_neuron_set = obi.PredefinedNeuronSet(node_set="All")
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100])
replay_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=25)
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100)
property_neuron_set = obi.PropertyNeuronSet(
    property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
                     obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})]
)
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="L6_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [ ]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_implicit')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

In [ ]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_implicit/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.StimNeurons.sample_percentage=50.0/neuron_sets.PropertyNeuronSet.property_filter=layer=1,2,3,synapse_class=INH/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/S1nonbarrel_neurons"]["node_ids"][:])
h5.close()

### Implicit node populations, but still using an extrinsic population for replay.

This is achieved using a hard coded VPM neuron set.

In [ ]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
sim_neuron_set = obi.PredefinedNeuronSet(node_set="All")
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100])
replay_neuron_set = obi.nbS1VPMInputs(sample_percentage=25)
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100)
property_neuron_set = obi.PropertyNeuronSet(
    property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
                     obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})]
)
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="VPM_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [ ]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

In [ ]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.StimNeurons.sample_percentage=50.0/neuron_sets.PropertyNeuronSet.property_filter=layer=1,2,3,synapse_class=INH/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/VPM"]["node_ids"][:])
h5.close()

In [ ]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded/run_scan_config.json")

In [ ]:
grid_scan_ds